## Backgound and problem
New York is the leading city of the world. According to Globalization and World Cities Research Network 2008,it is considered as an alfa ++ city togheter with London. With an estimated 2018 population of 8,398,748 distributed over a land area of about 302.6 square miles (784 km2), New York is also the most densely populated major city in the United States. 
In this set, moving to New York could be an interesting and fruitfull period of the life. 
As italian physician, I'd like to move to NY close to an hospital close to an italian restaurant 
The project will be focuse on find out the hospital in center of NY with a good (rating average >7) italian restaurant

## Data and strategy
To approach this problem, I need 3 different set of data:
- list of NY postal code, Boroughs, Neighborhoods along with their latitude and longitude
- list of hospital in NY
- list of italian restaurant

### Data source
- NY postal code, Boroughs, Neighborhoods along with their latitude and longitude
- hospital in NY: foursquare API
- italian restaurant: foursquare API

# Analysis

### Imported the required library

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Folium installed
Libraries imported.


In [2]:
CLIENT_ID = 'GYUXRRUIPW0QOOJXLVF0PI4OOLSNFPF0IA22IV2C3EQ5K41V' # your Foursquare ID
CLIENT_SECRET = '4VDSEZY2D15ZN2Z11XLVL0PYVI3ATWQKQ0VPDF04UQ02Q1CX' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GYUXRRUIPW0QOOJXLVF0PI4OOLSNFPF0IA22IV2C3EQ5K41V
CLIENT_SECRET:4VDSEZY2D15ZN2Z11XLVL0PYVI3ATWQKQ0VPDF04UQ02Q1CX


In [3]:
address = 'New York, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7127281 -74.0060152


In [4]:
def get_venues(lat,lng):
    
    #set variables
    radius=1000
    LIMIT=100
    CLIENT_ID = os.environ['CLIENT_ID'] # your Foursquare ID
    CLIENT_SECRET = os.environ['CLIENT_SECRET'] # your Foursquare Secret
    VERSION = '20180605' # Foursquare API version
    
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    
    # get all the data
    results = requests.get(url).json()
    venue_data=results["response"]['groups'][0]['items']
    venue_details=[]
    for row in venue_data:
        try:
            venue_id=row['venue']['id']
            venue_name=row['venue']['name']
            venue_category=row['venue']['categories'][0]['name']
            venue_details.append([venue_id,venue_name,venue_category])
        except KeyError:
            pass
        
    column_names=['ID','Name','Category']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df

In [5]:
def get_venue_details(venue_id):
        
    CLIENT_ID = os.environ['CLIENT_ID'] # your Foursquare ID
    CLIENT_SECRET = os.environ['CLIENT_SECRET'] # your Foursquare Secret
    VERSION = '20180605' # Foursquare API version
    
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            venue_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
    
    # get all the data
    results = requests.get(url).json()
    venue_data=results['response']['venue']
    venue_details=[]
    try:
        venue_id=venue_data['id']
        venue_name=venue_data['name']
        venue_likes=venue_data['likes']['count']
        venue_rating=venue_data['rating']
        venue_tips=venue_data['tips']['count']
        venue_details.append([venue_id,venue_name,venue_likes,venue_rating,venue_tips])
    except KeyError:
        pass
        
    column_names=['ID','Name','Likes','Rating','Tips']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df

In [6]:
def get_new_york_data():
    url='https://cocl.us/new_york_dataset'
    resp=requests.get(url).json()
    # all data is present in features label
    features=resp['features']
    
    # define the dataframe columns
    column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
    # instantiate the dataframe
    new_york_data = pd.DataFrame(columns=column_names)
    
    for data in features:
        borough = data['properties']['borough'] 
        neighborhood_name = data['properties']['name']
        
        neighborhood_latlon = data['geometry']['coordinates']
        neighborhood_lat = neighborhood_latlon[1]
        neighborhood_lon = neighborhood_latlon[0]
    
        new_york_data = new_york_data.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
    return new_york_data

In [7]:
new_york_data=get_new_york_data()

### Search for Hospital nearby to NY center

In [8]:
search_query = 'Hospital'
radius = 1000
print(search_query + ' .... OK!')

Hospital .... OK!


In [9]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=GYUXRRUIPW0QOOJXLVF0PI4OOLSNFPF0IA22IV2C3EQ5K41V&client_secret=4VDSEZY2D15ZN2Z11XLVL0PYVI3ATWQKQ0VPDF04UQ02Q1CX&ll=40.7127281,-74.0060152&v=20180604&query=Hospital&radius=1000&limit=30'

In [10]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d0f6b80348eba002a309650'},
 'response': {'venues': [{'id': '4a82ef0af964a52092f91fe3',
    'name': 'NewYork-Presbyterian-Lower Manhattan Hospital',
    'location': {'address': '170 William St',
     'crossStreet': 'at Gold St',
     'lat': 40.71066770948401,
     'lng': -74.0054107430059,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71066770948401,
       'lng': -74.0054107430059}],
     'distance': 234,
     'postalCode': '10038',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['170 William St (at Gold St)',
      'New York, NY 10038',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d196941735',
      'name': 'Hospital',
      'pluralName': 'Hospitals',
      'shortName': 'Hospital',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-156

In [11]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe_hospital= json_normalize(venues)


In [12]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe_hospital.columns if col.startswith('location.')] + ['id']
dataframe_hospital_filtered = dataframe_hospital.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_hospital_filtered['categories'] = dataframe_hospital_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_hospital_filtered.columns = [column.split('.')[-1] for column in dataframe_hospital_filtered.columns]

dataframe_hospital_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,NewYork-Presbyterian-Lower Manhattan Hospital,Hospital,170 William St,US,New York,United States,at Gold St,234,"[170 William St (at Gold St), New York, NY 100...","[{'label': 'display', 'lat': 40.71066770948401...",40.710668,-74.005411,10038,NY,4a82ef0af964a52092f91fe3
1,NY Presbyterian Hospital Lower Manhattan Hosp ER,Emergency Room,NaN,US,New York,United States,NaN,324,"[New York, NY, United States]","[{'label': 'display', 'lat': 40.70998454215684...",40.709985,-74.004720,NaN,NY,4bad1a23f964a520b62d3be3
2,Tribeca Soho Animal Hospital,Veterinarian,5 Lispenard St.,US,New York,United States,Near Corner of 6th Avenue and Lispenard,879,[5 Lispenard St. (Near Corner of 6th Avenue an...,"[{'label': 'display', 'lat': 40.72057803284648...",40.720578,-74.004869,10013,NY,4b1c0e79f964a5205c0124e3
3,Battery Park Veterinary Hospital,Veterinarian,"21 South End Ave., Ground Floor",US,New York,United States,W Thames St,1126,"[21 South End Ave., Ground Floor (W Thames St)...","[{'label': 'display', 'lat': 40.70816281460506...",40.708163,-74.017927,10280,NY,4bc3820374a9a593eba5d4f6
4,Fountain Pen Hospital,Miscellaneous Shop,10 Warren St,US,New York,United States,Warren Between Broadway and Church Streets,184,[10 Warren St (Warren Between Broadway and Chu...,"[{'label': 'display', 'lat': 40.71393396217314...",40.713934,-74.007508,10007,NY,4b0c4219f964a520ef3923e3
5,Beekman-Downtown Hospital Staff Residence,Residential Building (Apartment / Condo),69 Gold St,US,New York,United States,NaN,386,"[69 Gold St, New York, NY 10038, United States]","[{'label': 'display', 'lat': 40.70937208226082...",40.709372,-74.004819,10038,NY,4c61d85786b6be9ad4bb8a34
6,Seaport Animal Hospital,Veterinarian,80 Beekman St,US,New York,United States,Within South Bridge Towers,469,"[80 Beekman St (Within South Bridge Towers), N...","[{'label': 'display', 'lat': 40.708821, 'lng':...",40.708821,-74.003925,10038,NY,4fa02f86a17c7ac321fb70f1
7,Reade Street Animal Hospital,Veterinarian,NaN,US,New York,United States,NaN,574,"[New York, NY 10013, United States]","[{'label': 'display', 'lat': 40.71677, 'lng': ...",40.716770,-74.010240,10013,NY,5ca39ea12db4a9002c18996e
8,NewYork Presbyterian Lower manhattan Hospital ...,Doctor's Office,170 William St,US,New York,United States,NaN,259,"[170 William St, New York, NY 10038, United St...","[{'label': 'display', 'lat': 40.71045992353215...",40.710460,-74.005290,10038,NY,51dc559f498e34f68023c745
9,The Hospital,Brewery,260 Meserole Ave,US,Brooklyn,United States,Bushwick Place,428,"[260 Meserole Ave (Bushwick Place), Brooklyn, ...","[{'label': 'display', 'lat': 40.71184025487042...",40.711840,-74.010959,11206,NY,4c58d71904f9be9a3517ee60


In [13]:
venue_id = '4a82ef0af964a52092f91fe3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4a82ef0af964a52092f91fe3?client_id=GYUXRRUIPW0QOOJXLVF0PI4OOLSNFPF0IA22IV2C3EQ5K41V&client_secret=4VDSEZY2D15ZN2Z11XLVL0PYVI3ATWQKQ0VPDF04UQ02Q1CX&v=20180604'

In [14]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'likes', 'dislike', 'ok', 'venueRatingBlacklisted', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'description', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4a82ef0af964a52092f91fe3',
 'name': 'NewYork-Presbyterian-Lower Manhattan Hospital',
 'contact': {'phone': '2123125000',
  'formattedPhone': '(212) 312-5000',
  'twitter': 'nyphospital'},
 'location': {'address': '170 William St',
  'crossStreet': 'at Gold St',
  'lat': 40.71066770948401,
  'lng': -74.0054107430059,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.71066770948401,
    'lng': -74.0054107430059}],
  'postalCode': '10038',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['170 William St (at Gold St)',
   'New York, NY 10038',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/newyorkpresbyterianlower-manhattan-hospital/4a82ef0af964a52092f91fe3',
 'categories': [{'id': '4bf58dd8d48988d196941735',
   'name': 'Hospital',
   'pluralName': 'Hospitals',
   'shortName': 'Hospital',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_',
    'suffix': '.png'},
   'primar

In [15]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


# NewYork-Presbyterian-Lower Manhattan Hospital is the closest hospital to NY center

### Lets look for italian Restaurant nearby 

In [16]:
search_query = 'Italian'
radius = 10000
print(search_query + ' .... OK!')

Italian .... OK!


In [17]:
CLIENT_ID = 'GYUXRRUIPW0QOOJXLVF0PI4OOLSNFPF0IA22IV2C3EQ5K41V' # your Foursquare ID
CLIENT_SECRET = '4VDSEZY2D15ZN2Z11XLVL0PYVI3ATWQKQ0VPDF04UQ02Q1CX' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50

In [18]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=GYUXRRUIPW0QOOJXLVF0PI4OOLSNFPF0IA22IV2C3EQ5K41V&client_secret=4VDSEZY2D15ZN2Z11XLVL0PYVI3ATWQKQ0VPDF04UQ02Q1CX&ll=40.7127281,-74.0060152&v=20180604&query=Italian&radius=10000&limit=50'

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d0f6b815d891b002cb5abc5'},
 'response': {'venues': [{'id': '4a4560a7f964a5201aa81fe3',
    'name': "Harry's Italian Pizza Bar",
    'location': {'address': '2 Gold St',
     'crossStreet': 'at Maiden Ln.',
     'lat': 40.7076335455641,
     'lng': -74.00699585960496,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.7076335455641,
       'lng': -74.00699585960496}],
     'distance': 573,
     'postalCode': '10038',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['2 Gold St (at Maiden Ln.)',
      'New York, NY 10038',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d110941735',
      'name': 'Italian Restaurant',
      'pluralName': 'Italian Restaurants',
      'shortName': 'Italian',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
       'suffix': '.png'},
      'primary': True}],
    'delivery': {'id': '294543',


In [20]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",294543,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/harrys-italian-2...,False,4a4560a7f964a5201aa81fe3,2 Gold St,...,"[2 Gold St (at Maiden Ln.), New York, NY 10038...","[{'label': 'display', 'lat': 40.7076335455641,...",40.707634,-74.006996,NaN,10038,NY,Harry's Italian Pizza Bar,v-1561291649,77559825
1,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,4a74a36af964a520fede1fe3,260 Bleecker St,...,"[260 Bleecker St (at Cornelia St), New York, N...","[{'label': 'display', 'lat': 40.73111661087598...",40.731117,-74.003043,NaN,10014,NY,Faicco's Italian Specialties,v-1561291649,NaN
2,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",NaN,NaN,NaN,NaN,NaN,NaN,False,51e7310c498e639ed27062b1,57 W 57th St,...,"[57 W 57th St (at 6th Ave), New York, NY 10019...","[{'label': 'display', 'lat': 40.76447598357648...",40.764476,-73.977032,NaN,10019,NY,Quality Italian,v-1561291649,NaN
3,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",286355,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/pisillo-97-nassa...,False,528bf16711d2b7722da6b51c,97 Nassau St,...,"[97 Nassau St (at Ann St), New York, NY 10038,...","[{'label': 'display', 'lat': 40.71053008446078...",40.710530,-74.007526,NaN,10038,NY,Pisillo Italian Panini,v-1561291649,105457747
4,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",NaN,NaN,NaN,NaN,NaN,NaN,False,3fd66200f964a5209ee81ee3,200 W 44th St,...,"[200 W 44th St (btwn Broadway & 8th Ave), New ...","[{'label': 'display', 'lat': 40.7574973, 'lng'...",40.757497,-73.986779,NaN,10036,NY,Carmine’s Italian Restaurant,v-1561291649,NaN


In [21]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe_hospital.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Harry's Italian Pizza Bar,Italian Restaurant,2 Gold St,US,New York,United States,at Maiden Ln.,573,"[2 Gold St (at Maiden Ln.), New York, NY 10038...","[{'label': 'display', 'lat': 40.7076335455641,...",40.707634,-74.006996,10038,NY,4a4560a7f964a5201aa81fe3
1,Faicco's Italian Specialties,Sandwich Place,260 Bleecker St,US,New York,United States,at Cornelia St,2062,"[260 Bleecker St (at Cornelia St), New York, N...","[{'label': 'display', 'lat': 40.73111661087598...",40.731117,-74.003043,10014,NY,4a74a36af964a520fede1fe3
2,Quality Italian,Italian Restaurant,57 W 57th St,US,New York,United States,at 6th Ave,6257,"[57 W 57th St (at 6th Ave), New York, NY 10019...","[{'label': 'display', 'lat': 40.76447598357648...",40.764476,-73.977032,10019,NY,51e7310c498e639ed27062b1
3,Pisillo Italian Panini,Sandwich Place,97 Nassau St,US,New York,United States,at Ann St,275,"[97 Nassau St (at Ann St), New York, NY 10038,...","[{'label': 'display', 'lat': 40.71053008446078...",40.710530,-74.007526,10038,NY,528bf16711d2b7722da6b51c
4,Carmine’s Italian Restaurant,Italian Restaurant,200 W 44th St,US,New York,United States,btwn Broadway & 8th Ave,5241,"[200 W 44th St (btwn Broadway & 8th Ave), New ...","[{'label': 'display', 'lat': 40.7574973, 'lng'...",40.757497,-73.986779,10036,NY,3fd66200f964a5209ee81ee3
5,Tableside Italian Cook Shoppe,Italian Restaurant,345 E 6th St,US,New York,United States,1st and 2nd Ave,2253,"[345 E 6th St (1st and 2nd Ave), New York, NY ...","[{'label': 'display', 'lat': 40.72655617438330...",40.726556,-73.986514,10003,NY,58dd8ac3a8b75966fda9a547
6,Italian American Museum,History Museum,155 Mulberry St,US,New York,United States,at Grand St,1024,"[155 Mulberry St (at Grand St), New York, NY 1...","[{'label': 'display', 'lat': 40.71919065572268...",40.719191,-73.997376,10013,NY,4b9d4c67f964a5206ca136e3
7,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,US,New York,United States,NaN,786,"[225 Murray St, New York, NY 10282, United Sta...","[{'label': 'display', 'lat': 40.71521779064671...",40.715218,-74.014739,10282,NY,4fa862b3e4b0ebff2f749f06
8,Ralph's Famous Italian Ices,Ice Cream Shop,357 Graham Ave,US,Brooklyn,United States,btwn Metropolitan and Conselyea,5182,[357 Graham Ave (btwn Metropolitan and Consely...,"[{'label': 'display', 'lat': 40.71452172222155...",40.714522,-73.944636,11211,NY,4a7dd775f964a520d7ef1fe3
9,Pisillo Italian Cafe,Café,NaN,US,New York,United States,NaN,280,"[New York, NY, United States]","[{'label': 'display', 'lat': 40.71049280792626...",40.710493,-74.007546,NaN,NY,588a4316326c5a4b60559f17


In [22]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='NewYork-Presbyterian-Lower Manhattan Hospital',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [23]:
venue_id = '4a4560a7f964a5201aa81fe3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4a4560a7f964a5201aa81fe3?client_id=GYUXRRUIPW0QOOJXLVF0PI4OOLSNFPF0IA22IV2C3EQ5K41V&client_secret=4VDSEZY2D15ZN2Z11XLVL0PYVI3ATWQKQ0VPDF04UQ02Q1CX&v=20180604'

In [24]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'delivery', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'venuePage', 'reasons', 'page', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4a4560a7f964a5201aa81fe3',
 'name': "Harry's Italian Pizza Bar",
 'contact': {'phone': '2127470797',
  'formattedPhone': '(212) 747-0797',
  'twitter': 'harrysitalian'},
 'location': {'address': '2 Gold St',
  'crossStreet': 'at Maiden Ln.',
  'lat': 40.7076335455641,
  'lng': -74.00699585960496,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.7076335455641,
    'lng': -74.00699585960496}],
  'postalCode': '10038',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['2 Gold St (at Maiden Ln.)',
   'New York, NY 10038',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/harrys-italian-pizza-bar/4a4560a7f964a5201aa81fe3',
 'categories': [{'id': '4bf58dd8d48988d110941735',
   'name': 'Italian Restaurant',
   'pluralName': 'Italian Restaurants',
   'shortName': 'Italian',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
    'suffix': '.png'},
   'primary': True},
  {'id': '4bf58

In [25]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.0


# Harry's Italian Pizza Bar	is an italian Restaurant with a grade >7 ad it is close to NewYork-Presbyterian-Lower Manhattan Hospital. So I can move 